In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
batch_size=32

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 309787948.80it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 28021210.69it/s]


Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 114632467.50it/s]


Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20617455.38it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



In [10]:
d = iter(train_loader)
img, label = next(d)
print(img.shape)
print(label.shape)

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [12]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [18]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 500 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   500] loss: 0.011
[1,  1000] loss: 0.011
[1,  1500] loss: 0.015
[2,   500] loss: 0.015
[2,  1000] loss: 0.016
[2,  1500] loss: 0.017
[3,   500] loss: 0.014
[3,  1000] loss: 0.010
[3,  1500] loss: 0.017
[4,   500] loss: 0.014
[4,  1000] loss: 0.014
[4,  1500] loss: 0.016
[5,   500] loss: 0.009
[5,  1000] loss: 0.007
[5,  1500] loss: 0.014
[6,   500] loss: 0.011
[6,  1000] loss: 0.016
[6,  1500] loss: 0.014
[7,   500] loss: 0.006
[7,  1000] loss: 0.006
[7,  1500] loss: 0.021
[8,   500] loss: 0.010
[8,  1000] loss: 0.015
[8,  1500] loss: 0.016
[9,   500] loss: 0.012
[9,  1000] loss: 0.015
[9,  1500] loss: 0.013
[10,   500] loss: 0.010
[10,  1000] loss: 0.009
[10,  1500] loss: 0.008
Finished Training


In [19]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [20]:
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 97 %
